In [ ]:
import pandas as pd
from time import sleep
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


def f_score(cm, index = 2, beta = 1):
    tp = cm[index][index]
    fp = cm[:, index].sum() - cm[index][index]
    relevant = cm[index, :].sum()
    precision = tp/(tp + fp)
    recall = tp / (relevant)
    
    return (1 + beta ** 2) * (precision * recall)/(beta * beta * precision + recall)

    
    
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/uniform_sample/results/post_classification_random_sample_8-28 copy.csv'
df = pd.read_csv(path)

manual_column_name = 'alex'
pred_col = 'learned_classification'

if (manual_column_name not in set(df.columns)):
    df[manual_column_name] = [None for _ in range(len(df))]

options = {str(idx): el for idx, el in enumerate(['context', 'uses', 'extends'])} | {'*': '*'}
print(options)



In [ ]:
mask = df[manual_column_name].isna()
print(f"{mask.sum()}/{len(mask)} left.", flush = True)

In [ ]:
import random
from IPython.display import clear_output


mask = df[manual_column_name].isna()
while mask.sum() > 0:
    idx = random.choice(df[mask].index)
    
    print(f"{mask.sum()}/{len(mask)} left.", flush = True)
    print(options, flush = True)
    print(f"Model Key:{df.at[idx,'modelKey']}", flush = True)
    print(df.at[idx, 'multisentence'], flush = True)
    
    sleep(1)

    response = input()
    
    if (response == 'exit'):
        #df.to_csv(path) commenting out to avoid oopsies
        clear_output()
        break
    
    df.loc[idx, manual_column_name] = options.get(response)
    mask = df[manual_column_name].isna()
    clear_output()

    

In [ ]:
not_table = lambda s: s.find('\\begin{tabular}') == -1

confidence_threshold = 5
subset_mask = (df['alex'].notna() & df['multisentence'].apply(not_table)
               & (df['alex_confidence'] > confidence_threshold)
               #& (df['alex_confidence'] <= confidence_threshold)
               )

df_not_null = df[subset_mask]

true, pred = df_not_null['alex'], df_not_null[pred_col]

print(set(true), set(pred))

cm = confusion_matrix(true, pred)

use_weights = False
weights = np.array([0.92822272, 0.00804214, 0.06373514]).reshape(1, 3) * (10000 / cm.sum())
cm = np.round(cm * weights).astype(int) if use_weights else cm

y_label, x_label = 'alex', pred_col


labels = ['context', 'extends', 'uses']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel(x_label)
plt.ylabel(y_label)

print(f"Uses F1: {f_score(cm, index = 2, beta = 1)}")
print(f"Extends F1: {f_score(cm, index = 1)}")

In [ ]:
fp = df_not_null[(df_not_null[pred_col] == 'extends') & (df['alex'] == 'context')]
for sentence in fp['multisentence']:
    print(sentence, '\n')

In [ ]:
fp['multisentence'].apply(lambda s: s.find('\\begin{table}') > 0).sum()/len(fp)

In [ ]:
for idx, sentence in enumerate(df_not_null[df_not_null['alex'] == 'extends']['multisentence']):
    print(idx, sentence, '\n')

In [ ]:
df.columns